In [ ]:
import pandas as pd
import os
from datetime import datetime
import warnings
from pandas.tseries.offsets import DateOffset, MonthEnd
from openpyxl import load_workbook

warnings.simplefilter(action='ignore', category=UserWarning)

def calculate_lease_start(install_date, clause_type='monthly rule'):
    if pd.isnull(install_date):
        return None
    # ✅ Do not change lease start date if installation is on the 1st
    if install_date.day == 1:
        return install_date

    if clause_type == 'monthly rule':
        return datetime(install_date.year + (install_date.month // 12),
                        1 if install_date.month == 12 else install_date.month + 1,
                        1)
    elif clause_type == '2 weeks rule':
        if install_date.day <= 15:
            return datetime(install_date.year, install_date.month, 1)
        else:
            return datetime(install_date.year + (install_date.month // 12),
                            1 if install_date.month == 12 else install_date.month + 1,
                            1)
            print(f"Install date: {install_date} → Lease start: {lease_start}")

    return None

def get_fis_data(fis_file_path):
    header_df = pd.read_excel(fis_file_path, sheet_name="Header", header=None)
    billing_clause_value = None
    for row in header_df.itertuples(index=False):
        for col_idx, cell in enumerate(row):
            if isinstance(cell, str) and "billing start clause" in cell.lower():
                if col_idx + 1 < len(row):
                    billing_clause_value = row[col_idx + 1]
                break
        if billing_clause_value is not None:
            break

    billing_clause_value = str(billing_clause_value).strip().lower()
    if "monthly" in billing_clause_value:
        billing_clause_value = "monthly rule"
    elif "2 week" in billing_clause_value:
        billing_clause_value = "2 weeks rule"
    else:
        billing_clause_value = "monthly rule"  # Default

    fleet_df = pd.read_excel(fis_file_path, sheet_name="Fleet Data", header=5)
    fleet_df.columns = [str(col).strip() for col in fleet_df.columns]
    serial_col = next((col for col in fleet_df.columns if "serial" in col.lower()), None)
    
    ms_map = fleet_df.set_index(serial_col)["MS Contract ID"].to_dict() if "MS Contract ID" in fleet_df.columns else {}
    co_map = fleet_df.set_index(serial_col)["CO_ID"].to_dict() if "CO_ID" in fleet_df.columns else {}
    
    return billing_clause_value, ms_map, co_map

def get_asset_data(asset_file_path):
    preview = pd.read_excel(asset_file_path, sheet_name=0, nrows=10, header=None)
    header_row = None
    for i in range(len(preview)):
        row = preview.iloc[i].astype(str).str.lower()
        if row.str.contains("serial").any() and row.str.contains("install").any():
            header_row = i
            break
    if header_row is None:
        raise ValueError("Header row not found in asset file")

    df = pd.read_excel(asset_file_path, header=header_row)
    df.columns = [str(col).strip() for col in df.columns]

    serial_col = next((col for col in df.columns if "serial" in col.lower()), None)
    install_col = next((col for col in df.columns if "install" in col.lower()), None)
    mfg_col = next((col for col in df.columns if "routing" in col.lower() or "mfg pro" in col.lower()), None)

    df[install_col] = pd.to_datetime(df[install_col], errors='coerce')
    install_map = df.set_index(serial_col)[install_col].to_dict()
    mfg_map = df.set_index(serial_col)[mfg_col].to_dict() if mfg_col else {}

    return install_map, mfg_map

def read_clean_hardware_file(hw_path):
    raw = pd.read_excel(hw_path, header=None, engine='openpyxl')
    header_row = None
    for i in range(min(30, len(raw))):
        row = raw.iloc[i].astype(str).str.lower().fillna('')
        if row.str.contains("product").any() and row.str.contains("price").any():
            header_row = i
            break
    if header_row is None:
        raise ValueError("Header not found in hardware file")
    df = pd.read_excel(hw_path, header=header_row, engine='openpyxl')
    df.columns = df.columns.map(lambda x: str(x).replace('\n', ' ').strip())
    return df, raw

def calculate_lease_end(start_date_str, term_months):
    if pd.isnull(start_date_str) or pd.isnull(term_months):
        return ""
    try:
        start_date = pd.to_datetime(start_date_str, errors='coerce')
        if pd.isnull(start_date):
            return ""
        end_date = start_date + DateOffset(months=int(term_months) - 1)
        lease_end = end_date + MonthEnd(0)
        return lease_end.strftime('%m/%d/%Y')
    except:
        return ""

def process_single_file(filtered_path, asset_path, hw_path, main_folder, base_type):

    df = pd.read_excel(filtered_path)
    df.columns = [str(col).strip() for col in df.columns]

    lease_start_col = next((col for col in df.columns if str(col).lower().startswith("lease start date")), None)
    lease_end_col = next((col for col in df.columns if str(col).lower().startswith("lease end date")), None)
    if not lease_start_col:
        print(f"[SKIPPED] Lease start date column not found in {filtered_path}")
        return

   # Check sheet names to determine file type
    sheet_names = pd.ExcelFile(asset_path).sheet_names
    
    if "Header" in [s.strip().lower() for s in sheet_names] or "Fleet Data" in [s.strip().lower() for s in sheet_names]:
        billing_clause, ms_map, co_map = get_fis_data(asset_path)
    else:
        billing_clause = "monthly rule"  # Default fallback
        ms_map = {}
        co_map = {}
        
    install_map, mfg_map = get_asset_data(asset_path)


    df[lease_start_col] = df["SerialNumber"].map(lambda sn: install_map.get(sn))
    df[lease_start_col] = df[lease_start_col].apply(lambda d: calculate_lease_start(d, billing_clause))
    df[lease_start_col] = pd.to_datetime(df[lease_start_col], errors='coerce')
    df[lease_start_col] = df[lease_start_col].apply(lambda x: x.strftime('%m/%d/%Y') if pd.notnull(x) else "")

    def get_serial_column(columns):
        for option in ["SerialNumber", "Serial_No", "serialnumber", "serial_no"]:
            if option in columns:
                return option
        raise ValueError("Serial number column not found")

    if "fis" in base_type.lower():
        # Use raw FIS fleet dataframe again
        fis_fleet_df = pd.read_excel(asset_path, sheet_name="Fleet Data", header=5)
        fis_fleet_df.columns = [str(col).strip() for col in fis_fleet_df.columns]
    
        try:
            serial_col = get_serial_column(fis_fleet_df.columns)
    
            if "MS Contract ID" in fis_fleet_df.columns:
                ms_contract_map = fis_fleet_df.set_index(serial_col)["MS Contract ID"].to_dict()
                df["MFG Pro#"] = df["SerialNumber"].map(ms_contract_map)
            else:
                print(f"[WARNING] 'MS Contract ID' column not found in FIS file {asset_path}")
    
            if "CO_ID" in fis_fleet_df.columns:
                co_id_map = fis_fleet_df.set_index(serial_col)["CO_ID"].to_dict()
                df["Description"] = df["SerialNumber"].map(co_id_map)
            else:
                print(f"[WARNING] 'CO_ID' column not found in FIS file {asset_path}")
    
        except Exception as e:
            print(f"[ERROR] Failed to process FIS file: {e}")
    
    elif "dcc" in base_type.lower():
        if "PurchaseOrderNo" in df.columns:
            def extract_description(po):
                try:
                    parts = str(po).split("_")
                    for i, part in enumerate(parts):
                        if part == "CO" and i + 1 < len(parts):
                            return f"CO_{parts[i + 1]}"
                        elif part.startswith("CO_"):
                            return part
                    return ""
                except:
                    return ""
    
            def extract_mfg_pro(po):
                try:
                    parts = str(po).split("_")
                    for part in parts:
                        if len(part) == 8 and part.isalnum():
                            return part.upper()
                        # ✅ Case 2: 8-digit numeric (if ever occurs)
                        elif len(part) == 8 and part.isdigit():
                            return part
                            
                        # # ✅ Case 3: 10-character alphanumeric (if ever occurs)
                        # elif len(part) == 10 and part.isalnum():
                        #     return part.upper()
                    return ""
                except:
                    return ""
    
            df["Description"] = df["PurchaseOrderNo"].apply(extract_description)
            df["MFG Pro#"] = df["PurchaseOrderNo"].apply(extract_mfg_pro)
        else:
            df["MFG Pro#"] = ""
            df["Description"] = ""
            print(f"[WARNING] 'PurchaseOrderNo' column not found in file: {filtered_path}")




    hw_clean, hw_raw = read_clean_hardware_file(hw_path)

    term = coterm = None
    for row in hw_raw.itertuples(index=False):
        for idx, cell in enumerate(row):
            if isinstance(cell, str):
                if "pricing term" in cell.lower() and term is None:
                    term = row[idx + 1] if idx + 1 < len(row) else ""
                if "coterminous" in cell.lower() and coterm is None:
                    coterm = row[idx + 1] if idx + 1 < len(row) else ""
    df["Term"] = term
    df["Coterminous Y/N?"] = coterm if coterm else ""

    if lease_start_col and "Term" in df.columns:
        df[lease_end_col if lease_end_col else "Lease end date(mm/dd/yyyy)"] = df.apply(
            lambda row: calculate_lease_end(row[lease_start_col], row["Term"]),
            axis=1
        )
    if "ProductNumber" in df.columns and "NetUnitPrice" in df.columns:
        # Keep original ProductNumber in df
        df["_CleanProductNumber"] = (
            df["ProductNumber"].astype(str).str.split("#").str[0].str.strip().str.upper()
        )
    
        hw_clean["Product Number"] = (
            hw_clean.get("Product Number", "").astype(str).str.split("#").str[0].str.strip().str.upper()
        )
        hw_clean["Logistics Part Nbr"] = (
            hw_clean.get("Logistics Part Nbr", "").astype(str).str.split("#").str[0].str.strip().str.upper()
        )

        
    
        # Build a mapping from product to all possible prices
        from collections import defaultdict
        
        product_price_map = defaultdict(list)
        
        if "OPG / BD Net Price" in hw_clean.columns:
            hw_clean["OPG / BD Net Price"] = pd.to_numeric(hw_clean["OPG / BD Net Price"], errors='coerce')
            hw_clean["Copyright Fee"] = pd.to_numeric(hw_clean.get("Copyright Fee", 0), errors='coerce').fillna(0)
            hw_clean["calculated_price"] = hw_clean["OPG / BD Net Price"] + hw_clean["Copyright Fee"]
            for prod, price in hw_clean.set_index("Product Number")["calculated_price"].dropna().items():
                product_price_map[prod].append(price)
        
        if "Price" in hw_clean.columns:
            hw_clean["Price"] = pd.to_numeric(hw_clean["Price"], errors='coerce')
            for prod, price in hw_clean.set_index("Logistics Part Nbr")["Price"].dropna().items():
                product_price_map[prod].append(price)
        
        # Updated price match function
        def price_match_logic(row):
            product = row["_CleanProductNumber"]
            net_price = row["NetUnitPrice"]
            
            if pd.isnull(net_price):
                return ""
            
            possible_prices = product_price_map.get(product, [])
            
            # Print all available prices for debug
            # print(f"Product: {product}, NetUnitPrice: {net_price}, Hardware Prices: {possible_prices}")
            
            if not possible_prices:
                return "Product Number not found"
            
            # Check if any hardware price matches exactly
            for price in possible_prices:
                if abs(net_price - price) < 0.01:  # exact match within rounding tolerance
                    return "Match"
            
            return "Mismatch"
        
        # Apply to dataframe
        df["Price Match"] = df.apply(price_match_logic, axis=1)

  
        # Variance logic with "check next row if missing"
        # Collect lease price + LRF per row (not separately)
        lease_lrf_map = defaultdict(list)
        
        if "Leasing Net Price" in hw_clean.columns and "LRF" in hw_clean.columns:
            hw_clean["Leasing Net Price"] = pd.to_numeric(hw_clean["Leasing Net Price"], errors="coerce")
            hw_clean["LRF"] = pd.to_numeric(hw_clean["LRF"].ffill(), errors="coerce")  # forward fill
        
            for _, row in hw_clean.iterrows():
                lease_price = row.get("Leasing Net Price")
                lrf = row.get("LRF")
                prod1 = str(row.get("Product Number", "")).split("#")[0].strip().upper()
                prod2 = str(row.get("Logistics Part Nbr", "")).split("#")[0].strip().upper()
        
                if pd.notna(lease_price) and pd.notna(lrf) and lease_price != 0:
                    if prod1:
                        lease_lrf_map[prod1].append((lease_price, lrf))
                    if prod2:
                        lease_lrf_map[prod2].append((lease_price, lrf))

        
        def compute_variance(row):
            prod = row["_CleanProductNumber"]
            serial = row.get("SerialNumber", "")
            quantity = row.get("OrderedQuantity", 1)
        
            try:
                quantity = int(quantity)
            except:
                quantity = 1
        
            pairs = lease_lrf_map.get(prod, [])
        
            # Debug print
            #print(f"[DEBUG Variance] Product: {prod}, Serial: {serial}, Qty: {quantity}, Pairs: {pairs}")
        
            if not pairs or prod in {"ZD052A", "ZD053A", "ZD054A"}:
                return ""
        
            # Take the first available pair (you could also loop through if multiple exist)
            lease_price, lrf = pairs[0]
        
            if pd.isna(serial) or str(serial).strip() == "":
                return round(lease_price * lrf * quantity, 2)
            else:
                return round(lease_price * lrf, 2)

        df["Variance Amount"] = df.apply(compute_variance, axis=1)
        
        # Drop temp column before saving
        df.drop(columns=["_CleanProductNumber"], inplace=True)

        # Output folder logic
            # Output folder logic
        base_type_clean = base_type.replace(" ", "_").lower()  # "fis based" → "fis_based"
        output_folder_name = "output_fis" if "fis" in base_type_clean else "output_dcc"
        output_folder = os.path.join(main_folder, output_folder_name)
        os.makedirs(output_folder, exist_ok=True)
    
        file_name = os.path.basename(filtered_path)
        out_path = os.path.join(output_folder, file_name.replace(".xlsx", " checked.xlsx").replace(".xlsm", " checked.xlsm"))
    
        df.to_excel(out_path, index=False)
        print(f"[SAVED] Output written to: {out_path}")



def process_all_subfolders(main_folder):
    # Normalize folder names (case-insensitive match)
    all_base_folders = {folder.lower(): os.path.join(main_folder, folder)
                        for folder in os.listdir(main_folder)
                        if os.path.isdir(os.path.join(main_folder, folder))}

    base_folders = {
        "fis based": all_base_folders.get("fis based"),
        "dcc based": all_base_folders.get("dcc based")
    }

    for base_type, base_path in base_folders.items():
        if not base_path:
            print(f"[SKIPPED] '{base_type}' folder not found in main folder.")
            continue

        for subfolder in os.listdir(base_path):
            subfolder_path = os.path.join(base_path, subfolder)
            if not os.path.isdir(subfolder_path):
                continue

            print(f"\n[PROCESSING] {base_type} → Sample Folder: {subfolder}")
            filtered_file = None
            hardware_file = None
            asset_or_fis_file = None

            for file in os.listdir(subfolder_path):
                file_lower = file.lower()
            
                if any(x in file_lower for x in ["temp exhibit", "temp_exhibit"]) and file_lower.endswith(('.xlsx', '.xlsm')):
                    filtered_file = os.path.join(subfolder_path, file)
            
                elif "supplychainoutput-hardware" in file_lower and file_lower.endswith(('.xlsx', '.xlsm')):
                    hardware_file = os.path.join(subfolder_path, file)
            
                elif file_lower.endswith(('.xlsx', '.xlsm')):
                    if base_type == "fis based" and "fis" in file_lower:
                        asset_or_fis_file = os.path.join(subfolder_path, file)
                    elif base_type == "dcc based" and "asset" in file_lower:
                        asset_or_fis_file = os.path.join(subfolder_path, file)


            if not (filtered_file and hardware_file and asset_or_fis_file):
                print(f"[SKIPPED] Missing one or more required files in: {subfolder_path}")
                continue

            try:
                process_single_file(
                    filtered_file,
                    asset_or_fis_file,
                    hardware_file,
                    main_folder,
                    base_type
                )
            except Exception as e:
                print(f"[ERROR] Failed processing {filtered_file}: {e}")

# Update this path as needed

main_folder = r"C:\Users\AtCh030\Desktop\COI-COA Bot Input\September testing_2nd round\September testing_2nd round"
process_all_subfolders(main_folder)



[PROCESSING] fis based → Sample Folder: Sample 1
[SAVED] Output written to: C:\Users\AtCh030\Desktop\COI-COA Bot Input\September testing_2nd round\September testing_2nd round\output_fis\Temp_Exhibit_ES_Kuhne&Nagel_29.09.2025 checked.xlsx

[PROCESSING] fis based → Sample Folder: Sample 2
[SAVED] Output written to: C:\Users\AtCh030\Desktop\COI-COA Bot Input\September testing_2nd round\September testing_2nd round\output_fis\Temp_Exhibit_DE_Cargill_30.09.2025 checked.xlsx

[PROCESSING] fis based → Sample Folder: Sample 3
[SAVED] Output written to: C:\Users\AtCh030\Desktop\COI-COA Bot Input\September testing_2nd round\September testing_2nd round\output_fis\Temp_Exhibit_FR_Renault_30.09.2025 checked.xlsx

[PROCESSING] fis based → Sample Folder: Sample 4
[SAVED] Output written to: C:\Users\AtCh030\Desktop\COI-COA Bot Input\September testing_2nd round\September testing_2nd round\output_fis\Temp_exhibit_BE_LIDL_29.09.2025 checked.xlsx

[PROCESSING] dcc based → Sample Folder: Sample 1
[SAVED] O